# Spike Detection (이상 급증 탐지)

이 노트북은 `BaselineAggregator` 모듈을 사용하여 키워드별 스파이크를 탐지합니다.

## 주요 기능
1. **베이스라인 집계**: 최근 구간과 기준 구간 비교
2. **다중 탐지 방법**: Ratio, Z-score, Poisson 검정
3. **앙상블 판정**: 여러 방법의 결과를 조합
4. **다중검정 보정**: FDR-BH 방법으로 False Positive 감소

## 윈도우 설정
- **Window 1**: recent=[당월], base=[전월]
- **Window 3**: recent=[당월,-1,-2], base=[-3,-4,-5] (겹치지 않음)

## 1. 환경 설정

In [ ]:
# =====================
# 표준 라이브러리
# =====================
import sys
from pathlib import Path

# =====================
# 서드 파티 라이브러리
# =====================
import polars as pl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns

# =====================
# 경로 설정
# =====================
PROJECT_ROOT = Path.cwd().parent
DATA_DIR = PROJECT_ROOT / "data"
FONT_DIR = PROJECT_ROOT / 'font'

# Python 내장 src 모듈과의 충돌 방지
if str(PROJECT_ROOT) in sys.path:
    sys.path.remove(str(PROJECT_ROOT))
sys.path.insert(0, str(PROJECT_ROOT))


# =====================
# 로컬 모듈
# =====================
from src.loading import DataLoader
from src.utils.baseline_aggregator import BaselineAggregator

# =====================
# 시각화 설정
# =====================
font_path = FONT_DIR / 'PretendardVariable.ttf'
fm.fontManager.addfont(str(font_path))
font_prop = fm.FontProperties(fname=font_path)
plt.rcParams['font.family'] = font_prop.get_name()
plt.rcParams["axes.unicode_minus"] = False
sns.set_style("whitegrid")

print("✅ 모듈 import 완료")


## 2. 데이터 로드

In [ ]:
# 데이터 로딩
loader = DataLoader(
    output_file=DATA_DIR / 'silver' / 'maude_clustered.parquet'
)

# Polars 어댑터 설정
adapter = 'polars'
polars_kwargs = {
    'use_statistics': True,  # 통계 정보 활용
    'parallel': 'auto',      # 자동 병렬 처리
    'low_memory': False,     # 메모리 최적화 비활성화 (성능 우선)
    'rechunk': False,        # 데이터 재정렬 비활성화
    'cache': True,           # 중간 결과 캐싱 활성화
}

# LazyFrame으로 로드 (실제 연산은 collect() 호출 시 수행)
lf = loader.load(adapter=adapter, **polars_kwargs)

In [ ]:
# 기본 정보 확인
total_rows = lf.select(pl.len()).collect().item()
print(f"전체 행 수: {total_rows:,}개")

# 스키마 확인
print("\n스키마:")
lf.collect_schema()

## 3. BaselineAggregator 초기화

In [ ]:
# Aggregator 초기화
aggregator = BaselineAggregator(lf)

print("✅ BaselineAggregator 초기화 완료")

## 4. 베이스라인 테이블 생성

모든 키워드에 대해 다음을 계산합니다:
- `C_recent`, `C_base`: 최근/기준 구간의 키워드 보고서 수
- `N_recent`, `N_base`: 최근/기준 구간의 전체 보고서 수
- `ratio`: 증가율 (C_recent+1) / (C_base+1)
- `score_ratio`, `score_log`, `score_sqrt`: 가중 점수
- `z_log`: log 변환 후 z-score
- `p_pois`, `p_adjusted`: Poisson 검정 p-value (다중검정 보정)
- `is_spike`, `is_spike_z`, `is_spike_p`: 각 방법별 스파이크 여부
- `pattern`: 앙상블 판정 (severe/alert/attention/general)

In [ ]:
# 베이스라인 테이블 생성
baseline_lf = aggregator.create_baseline_table(
    as_of_month=None,              # None = 자동으로 최신 월 사용
    z_threshold=2.0,                # z-score 임계값
    min_c_recent=20,                # 최소 보고서 수 (노이즈 필터링)
    eps=0.1,                        # z-score 분모 보정값
    alpha=0.05,                     # Poisson 유의수준
    correction_method="fdr_bh",    # 다중검정 보정 (fdr_bh, bonferroni, sidak, None)
    verbose=True                    # 진행 상황 출력
)

## 5. 베이스라인 테이블 조회

In [ ]:
# 전체 테이블 확인 (상위 10개)
baseline_df = baseline_lf.collect()
print(f"총 키워드 수: {baseline_df['keyword'].n_unique()}")
print(f"총 행 수: {len(baseline_df)} (윈도우 1개월 + 3개월)")

baseline_df.head(10)

In [ ]:
# Window 1 결과만 보기 (score_ratio 기준 정렬)
baseline_lf.filter(
    pl.col("window") == 1
).sort(
    "score_ratio", descending=True
).head(20).collect()

In [ ]:
# Window 3 결과만 보기
baseline_lf.filter(
    pl.col("window") == 3
).sort(
    "score_ratio", descending=True
).head(20).collect()

## 6. 스파이크 탐지 - 방법별

### 6.1 Ratio 기반 탐지
상대적 증가율 기반. `score_ratio = log(ratio) * log(C_recent+1)`

In [ ]:
# Ratio 기반 스파이크 (Window 1)
spikes_ratio_w1 = aggregator.detect_spikes(
    baseline_lf, 
    window=1, 
    spike_type="ratio"
).collect()

print(f"Ratio 기반 스파이크 (Window 1): {len(spikes_ratio_w1)}개")
spikes_ratio_w1.head(10)

In [ ]:
# Ratio 기반 스파이크 (Window 3)
spikes_ratio_w3 = aggregator.detect_spikes(
    baseline_lf, 
    window=3, 
    spike_type="ratio"
).collect()

print(f"Ratio 기반 스파이크 (Window 3): {len(spikes_ratio_w3)}개")
spikes_ratio_w3.head(10)

### 6.2 Z-score 기반 탐지
통계적 유의성 기반. `z_log >= 2.0`

In [ ]:
# Z-score 기반 스파이크 (Window 1)
spikes_z_w1 = aggregator.detect_spikes(
    baseline_lf, 
    window=1, 
    spike_type="z"
).collect()

print(f"Z-score 기반 스파이크 (Window 1): {len(spikes_z_w1)}개")
spikes_z_w1.head(10)

In [ ]:
# Z-score 기반 스파이크 (Window 3)
spikes_z_w3 = aggregator.detect_spikes(
    baseline_lf, 
    window=3, 
    spike_type="z"
).collect()

print(f"Z-score 기반 스파이크 (Window 3): {len(spikes_z_w3)}개")
spikes_z_w3.head(10)

### 6.3 Poisson 기반 탐지
확률적 유의성 기반. `p_adjusted <= 0.05`

In [ ]:
# Poisson 기반 스파이크 (Window 1)
spikes_poisson_w1 = aggregator.detect_spikes(
    baseline_lf, 
    window=1, 
    spike_type="poisson"
).collect()

print(f"Poisson 기반 스파이크 (Window 1): {len(spikes_poisson_w1)}개")
spikes_poisson_w1.head(10)

In [ ]:
# Poisson 기반 스파이크 (Window 3)
spikes_poisson_w3 = aggregator.detect_spikes(
    baseline_lf, 
    window=3, 
    spike_type="poisson"
).collect()

print(f"Poisson 기반 스파이크 (Window 3): {len(spikes_poisson_w3)}개")
spikes_poisson_w3.head(10)

## 7. 앙상블 스파이크 탐지

### 앙상블 판정 기준
- **severe**: 3개 방법 모두 탐지
- **alert**: 2개 방법 탐지
- **attention**: 1개 방법 탐지
- **general**: 0개 방법 탐지

In [ ]:
# 앙상블 스파이크 (Window 1, min_methods=2: severe + alert)
spikes_ensemble_w1 = aggregator.detect_spike_ensemble(
    baseline_lf,
    window=1,
    min_methods=2  # 2개 이상 방법이 탐지한 것만
).collect()

print(f"앙상블 스파이크 (Window 1, min_methods≥2): {len(spikes_ensemble_w1)}개")
spikes_ensemble_w1.head(20)

In [ ]:
# 앙상블 스파이크 (Window 3, min_methods=2)
spikes_ensemble_w3 = aggregator.detect_spike_ensemble(
    baseline_lf,
    window=3,
    min_methods=2
).collect()

print(f"앙상블 스파이크 (Window 3, min_methods≥2): {len(spikes_ensemble_w3)}개")
spikes_ensemble_w3.head(20)

In [ ]:
# Severe만 조회 (3개 방법 모두 탐지)
severe_only_w1 = baseline_lf.filter(
    (pl.col("window") == 1) & 
    (pl.col("pattern") == "severe")
).sort("score_pois", descending=True).collect()

print(f"Severe 스파이크 (Window 1): {len(severe_only_w1)}개")
severe_only_w1

## 8. 패턴별 통계

In [ ]:
# 윈도우별 패턴 분포
pattern_stats = baseline_lf.group_by(["window", "pattern"]).agg(
    pl.len().alias("count")
).sort(["window", "pattern"]).collect()

print("패턴별 통계:")
pattern_stats

In [ ]:
# 패턴별 평균 지표 비교 (Window 1)
pattern_metrics = baseline_lf.filter(
    pl.col("window") == 1
).group_by("pattern").agg([
    pl.len().alias("count"),
    pl.col("C_recent").mean().round(2).alias("avg_C_recent"),
    pl.col("ratio").mean().round(2).alias("avg_ratio"),
    pl.col("z_log").mean().round(2).alias("avg_z_log"),
    pl.col("score_pois").mean().round(2).alias("avg_score_pois")
]).sort("pattern").collect()

print("패턴별 평균 지표 (Window 1):")
pattern_metrics

## 9. 시각화

In [ ]:
# 패턴별 분포 시각화
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for idx, window in enumerate([1, 3]):
    data = pattern_stats.filter(pl.col("window") == window).to_pandas()
    
    ax = axes[idx]
    ax.bar(data["pattern"], data["count"], 
           color=["red", "orange", "yellow", "green"])
    ax.set_title(f"Window {window} - 패턴별 키워드 수", fontsize=14, weight="bold")
    ax.set_xlabel("Pattern", fontsize=12)
    ax.set_ylabel("Count", fontsize=12)
    ax.grid(axis="y", alpha=0.3)
    
    # 값 표시
    for i, (pattern, count) in enumerate(zip(data["pattern"], data["count"])):
        ax.text(i, count + 0.5, str(count), ha="center", va="bottom", fontsize=11)

plt.tight_layout()
plt.show()

In [ ]:
# Severe/Alert 키워드들의 scatter plot (C_recent vs ratio)
w1_data = baseline_lf.filter(
    (pl.col("window") == 1) & 
    (pl.col("pattern").is_in(["severe", "alert"]))
).collect().to_pandas()

fig, ax = plt.subplots(figsize=(10, 6))

colors = {"severe": "red", "alert": "orange"}
for pattern in ["severe", "alert"]:
    subset = w1_data[w1_data["pattern"] == pattern]
    ax.scatter(subset["C_recent"], subset["ratio"], 
               label=pattern, alpha=0.6, s=100, color=colors[pattern])

ax.set_xlabel("C_recent (최근 보고서 수)", fontsize=12)
ax.set_ylabel("Ratio (증가율)", fontsize=12)
ax.set_title("Spike Keywords (Window 1): C_recent vs Ratio", fontsize=14, weight="bold")
ax.legend()
ax.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print(f"총 {len(w1_data)}개 키워드 (severe: {len(w1_data[w1_data['pattern']=='severe'])}, alert: {len(w1_data[w1_data['pattern']=='alert'])})")

## 10. 특정 키워드 분석

In [ ]:
# 특정 키워드 상세 조회 (예시)
keyword_to_check = "Electrical/Power"  # 원하는 키워드로 변경

keyword_detail = baseline_lf.filter(
    pl.col("keyword") == keyword_to_check
).collect()

print(f"키워드: {keyword_to_check}")
keyword_detail

## 11. 결과 저장 (옵션)

In [ ]:
# 베이스라인 테이블 전체 저장
output_dir = Path("../output")
output_dir.mkdir(exist_ok=True)

baseline_df.write_parquet(output_dir / "baseline_table.parquet")
print(f"✅ 베이스라인 테이블 저장 완료: {output_dir / 'baseline_table.parquet'}")

In [ ]:
# 앙상블 스파이크만 CSV로 저장
spikes_ensemble_w1.write_csv(output_dir / "spikes_ensemble_w1.csv")
spikes_ensemble_w3.write_csv(output_dir / "spikes_ensemble_w3.csv")

print(f"✅ 앙상블 스파이크 저장 완료")
print(f"  - Window 1: {output_dir / 'spikes_ensemble_w1.csv'}")
print(f"  - Window 3: {output_dir / 'spikes_ensemble_w3.csv'}")

## 12. 요약

이 노트북에서는:
1. ✅ `BaselineAggregator` 모듈을 사용하여 베이스라인 테이블 생성
2. ✅ Ratio, Z-score, Poisson 3가지 방법으로 스파이크 탐지
3. ✅ 앙상블 방식으로 robust한 스파이크 판정
4. ✅ 다중검정 보정으로 False Positive 최소화
5. ✅ 시각화 및 분석 결과 저장

### 다음 단계
- 탐지된 스파이크의 원인 분석 (제품 코드, 이벤트 텍스트 등)
- 시계열 추이 분석
- 알림 시스템 구축